# Imports

In [19]:
import pandas as pd
import numpy as np


# Data Understanding

In [3]:
injury_record = pd.read_csv('./data/InjuryRecord.csv')

In [4]:
injury_record.head(5)

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
0,39873,39873-4,39873-4-32,Knee,Synthetic,1,1,1,1
1,46074,46074-7,46074-7-26,Knee,Natural,1,1,0,0
2,36557,36557-1,36557-1-70,Ankle,Synthetic,1,1,1,1
3,46646,46646-3,46646-3-30,Ankle,Natural,1,0,0,0
4,43532,43532-5,43532-5-69,Ankle,Synthetic,1,1,1,1


In [5]:
injury_record.shape

(105, 9)

# EDA

### No of players with injuries

In [7]:
injury_record['PlayerKey'].nunique()

100

### Distribution of injuries across different surfaces

In [8]:
injury_record.groupby('Surface')['PlayerKey'].count()

Surface
Natural      48
Synthetic    57
Name: PlayerKey, dtype: int64

### Injuries for different body parts

In [9]:
injury_record.groupby('BodyPart')['PlayerKey'].count()

BodyPart
Ankle    42
Foot      7
Heel      1
Knee     48
Toes      7
Name: PlayerKey, dtype: int64

In [10]:
injury_record[injury_record['BodyPart'].isin(['Ankle','Knee'])].groupby(['BodyPart', 'Surface'])['PlayerKey'].count()

BodyPart  Surface  
Ankle     Natural      17
          Synthetic    25
Knee      Natural      24
          Synthetic    24
Name: PlayerKey, dtype: int64

In [11]:
injury_record['severity'] = injury_record['DM_M1'] + injury_record['DM_M7'] + injury_record['DM_M28'] + injury_record['DM_M42']

In [12]:
injury_record.groupby('severity')['PlayerKey'].count()

severity
1    29
2    39
3     8
4    29
Name: PlayerKey, dtype: int64

In [59]:
injury_record_agg = injury_record.groupby(['Surface','BodyPart','severity'])['PlayerKey'].count().\
                                reset_index()

In [62]:
bodypart_totalinjuries = injury_record_agg.groupby(['BodyPart','Surface'])['PlayerKey'].sum().\
                                reset_index().rename(columns = {'PlayerKey':'TotalInjuries'})

In [65]:
injury_record_agg_merged = pd.merge(injury_record_agg, 
                                    bodypart_totalinjuries, 
                                    left_on = ['BodyPart','Surface'], 
                                    right_on = ['BodyPart','Surface'])
injury_record_agg_merged['%injured'] = injury_record_agg_merged['PlayerKey']/injury_record_agg_merged['TotalInjuries']

In [66]:
injury_record_agg_merged.sort_values(by=['BodyPart', 'severity','Surface'], ascending=['True','False','True'])

,Surface,BodyPart,severity,PlayerKey,TotalInjuries,%injured
0,Natural,Ankle,1,8,17,0.470588
11,Synthetic,Ankle,1,8,25,0.320000
1,Natural,Ankle,2,6,17,0.352941
12,Synthetic,Ankle,2,7,25,0.280000
13,Synthetic,Ankle,3,2,25,0.080000
2,Natural,Ankle,4,3,17,0.176471
14,Synthetic,Ankle,4,8,25,0.320000
3,Natural,Foot,3,1,5,0.200000
15,Synthetic,Foot,3,1,2,0.500000
4,Natural,Foot,4,4,5,0.800000
